In [1]:
# Tạo folder data để các bạn upload file 3 file data được tạo ra ở step 0 vào folder data
# Các bạn upload file hand_gesture.yaml ngoài folder data
!mkdir data/

In [2]:
# Nếu đã chạy thành công step 0 thì các bạn không chạy cell này và chỉ upload 3 file data của các bạn vào thư mục data
# Và upload file hand_gesture.yaml bên ngoài thư mục data
# Nếu không chạy được ở step 0 thì các bạn run cell này để download data
!gdown  1gzWOtABiVmJ38usCSDe5F9gR2tECt3zu -O data/
!gdown  15lwipssmC_K82ukRfb0uVCiDH1TZ3QCf -O data/
!gdown  1nIo1_wBmkovz-u_BCsV5c1Kbz6ZqoKwq -O data/
# Download file hand_gesture.yam
!gdown  1ZteHYSgbuZu_GcUJHW8ZzoZv1DE8-oLw

Downloading...
From: https://drive.google.com/uc?id=1gzWOtABiVmJ38usCSDe5F9gR2tECt3zu
To: /content/data/landmark_val.csv
100% 369k/369k [00:00<00:00, 56.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=15lwipssmC_K82ukRfb0uVCiDH1TZ3QCf
To: /content/data/landmark_train.csv
100% 1.28M/1.28M [00:00<00:00, 144MB/s]
Downloading...
From: https://drive.google.com/uc?id=1nIo1_wBmkovz-u_BCsV5c1Kbz6ZqoKwq
To: /content/data/landmark_test.csv
100% 320k/320k [00:00<00:00, 125MB/s]
Downloading...
From: https://drive.google.com/uc?id=1ZteHYSgbuZu_GcUJHW8ZzoZv1DE8-oLw
To: /content/hand_gesture.yaml
100% 120/120 [00:00<00:00, 709kB/s]


In [3]:
!pip install mediapipe==0.10.18

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.1/36.1 MB 48.8 MB/s eta 0:00:00


In [4]:
!pip install torchmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 926.4/926.4 kB 21.8 MB/s eta 0:00:00


In [5]:
import os
import cv2
import yaml
import torch
import numpy as np
import pandas as pd
from  torch import nn
import mediapipe as mp
from torch import optim
from datetime import datetime
from torchmetrics import Accuracy
from torch.utils.data import Dataset

In [8]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        list_label = label_dict_from_config_file("hand_gesture.yaml")
        ################## Q1: Xây dựng kiến trúc Neural Network ################## Q1
        '''
        Hoàn thành đoạn code để xây dựng một model gồm có 4 hidden layer,
        lần lượng input và output là (63, 128), (128, 128), (128, 128), (128, 128).
        Layer đầu tiên được theo sau bổi một Relu và Batchnorm1d.
        Layer thứ 2, 3, và 4 được theo sau bỏi Relu và Dropout với rate lần lượt là 0.4, 0.4, 0.6.
        Output layer có nhịêm vụ phân loại với input là 128 và output là số lượng class cử chỉ
        '''
        ####################################################
        # nn.Sequential(*args) xây dựng mô hình một cách 'tuần tự'; có thể "batchNorm, Dropout"; không thể "skip connection"
        self.linear_relu_stack = nn.Sequential(
            # Layer 1: input 63 -> 128
            nn.Linear(63, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128),

            # Layer 2: 128 -> 128
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Dropout(0.4),

            # Layer 3: 128 -> 128
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Dropout(0.4),

            # Layer 4: 128 -> 128
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Dropout(0.6),

            # Output layer: 128 -> num_classes
            nn.Linear(128, len(list_label))
        )
    def forward(self, x):
        ################## Q2: Hàm forward ################## Q2
        ''' Hoàn thành code để thực hiện forward dự đoán cử chỉ với input x.
        Thực hiệnt flatten x
        Pass x vừa flatten vào linear_relu_stack
        Return  logits (outputs từ layer cuối cùng)
        '''

        ####################################################
        x = self.flatten(x)  # Làm phẳng input
        logits = self.linear_relu_stack(x)  # Đưa qua các layer
        return logits

    def predict(self,x,threshold=0.8):
        logits = self(x)
        softmax_prob = nn.Softmax(dim=1)(logits) # dim = 1 -> chiều softmax tính toán. VD: [1.0, 2.0, 3.0], [4,...].. -> e^1/(e^1+e^2+e^3)
        chosen_ind = torch.argmax(softmax_prob,dim=1) # dim = 1 -> chiều argmax xác định index_max.
        return torch.where(softmax_prob[0,chosen_ind]>threshold,chosen_ind,-1)

    def predict_with_known_class(self,x):
        logits = self(x)
        softmax_prob = nn.Softmax(dim=1)(logits)
        return torch.argmax(softmax_prob,dim=1)

    def score(self,logits):
        return -torch.amax(logits,dim=1)

In [9]:
def label_dict_from_config_file(relative_path):
    with open(relative_path,"r") as f:
       label_tag = yaml.full_load(f)["gestures"]
    return label_tag

In [10]:
class HandLandmarksDetector():
    def __init__(self) -> None:
        self.mp_drawing = mp.solutions.drawing_utils
        self.mp_drawing_styles = mp.solutions.drawing_styles
        self.mp_hands = mp.solutions.hands
        self.detector = self.mp_hands.Hands(False,max_num_hands=1,min_detection_confidence=0.5)

    def detectHand(self,frame):
        hands = []
        frame = cv2.flip(frame, 1)
        annotated_image = frame.copy()
        results = self.detector.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        if results.multi_hand_landmarks is not None:
            for hand_landmarks in results.multi_hand_landmarks:
                hand = []
                self.mp_drawing.draw_landmarks(
                    annotated_image,
                    hand_landmarks,
                    self.mp_hands.HAND_CONNECTIONS,
                    self.mp_drawing_styles.get_default_hand_landmarks_style(),
                    self.mp_drawing_styles.get_default_hand_connections_style())
                for landmark in hand_landmarks.landmark:
                    x,y,z = landmark.x,landmark.y,landmark.z
                    hand.extend([x,y,z])
            hands.append(hand)
        return hands,annotated_image

In [11]:
class CustomImageDataset(Dataset):
    def __init__(self, data_file):
        self.data = pd.read_csv(data_file)
        self.labels = torch.from_numpy(self.data.iloc[:,0].to_numpy())

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        one_hot_label = self.labels[idx]
        torch_data = torch.from_numpy(self.data.iloc[idx,1:].to_numpy(dtype=np.float32))
        return torch_data, one_hot_label

In [13]:
class EarlyStopper:
    def __init__(self, patience=1, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.watched_metrics = np.inf

    def early_stop(self, current_value):
        if current_value < self.watched_metrics:
            self.watched_metrics = current_value
            self.counter = 0
        elif current_value > (self.watched_metrics + self.min_delta):
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False

In [14]:
def train(trainloader, val_loader, model, loss_function, early_stopper, optimizer):
    # add auroc score
    best_vloss = 1_000_000
    timestamp = datetime.now().strftime('%d-%m %H:%M')
    for epoch in range(300):
        #training step
        model.train(True)
        running_loss = 0.0
        acc_train = Accuracy(num_classes=len(LIST_LABEL), task='MULTICLASS') # Accuracy in torchmetrics -> return Tensor
        for batch_number,data in enumerate(trainloader):
            inputs,labels = data # Load batch data

            ################## Q9. Reset gradients và dự đoán ################## Q9
            ''' Hoàn thành code để thực hiện reset gradients và dự đoán class cử
            chỉ của inputs
            '''
            # Reset gradients trước mỗi batch rất quan trọng vì PyTorch tích lũy gradients
            optimizer.zero_grad()  # Xóa gradients từ lần backward trước

            # Dự đoán class của cử chỉ
            preds = model(inputs)  # Forward pass qua model
            ####################################################

            ################## Q10: Tính loss và cập nhật model ################## Q10
            ''' Hoàn thành code để thực hiện tính loss dưa vào kết quả dự đoán
            và labels, sau đó thực hiện backwward và update parameters thông qua
            optimizer
            '''
            # Tính loss
            loss = loss_function(preds, labels)

            if batch_number == 1: # print test loss
                print(f"Loss batch_number == 1: type = {type(loss)} \n item: \n {loss.item()}")

            # Backward và update parameters
            loss.backward()  # Tính toán gradients
            optimizer.step() # Cập nhật parameters


            ####################################################

            acc_train.update(model.predict_with_known_class(inputs), labels)
            running_loss += loss.item()

        avg_loss = running_loss / len(trainloader) # Tính toán metrics cuối epoch

        # validating step
        model.train(False) # model.eval() # cùng ý nghĩa
        running_vloss = 0.0
        acc_val = Accuracy(num_classes=len(LIST_LABEL), task='MULTICLASS')
        for i, vdata in enumerate(val_loader):
            vinputs, vlabels = vdata
            preds = model(vinputs)
            vloss = loss_function(preds, vlabels)
            running_vloss += vloss.item()
            acc_val.update(model.predict_with_known_class(vinputs), vlabels)

        # Log the running loss averaged per batch
        # for both training and validation
        print(f"Epoch {epoch}: ")
        print(f"Accuracy train:{acc_train.compute().item()}, val:{acc_val.compute().item()}")
        avg_vloss = running_vloss / len(val_loader)
        print('LOSS train {} valid {}'.format(avg_loss, avg_vloss))
        print('Training vs. Validation Loss',
                        { 'Training' : avg_loss, 'Validation' : avg_vloss },
                        epoch + 1)
        print('Training vs. Validation accuracy',
                        { 'Training' : acc_train.compute().item()
                        , 'Validation' : acc_val.compute().item() },
                        epoch + 1)

        # Track best performance, and save the model's state
        if avg_vloss < best_vloss:
            best_vloss = avg_vloss
            best_model_path = f'./{save_path}/model_{timestamp}_{model.__class__.__name__}_best'
            torch.save(model.state_dict(), best_model_path)

        if early_stopper.early_stop(avg_vloss):
            ################## Q5: Early stopping monitoring ################## Q5
            ''' Hoàn thành đoạn code bên dướ để  print ra epoch hiện tại và
            minimum watched metric và thoát loop
            '''
            print(f"Early stopping at epoch {epoch}")
            print(f"Minimum watched metric: {early_stopper.watched_metrics}")
            break
            ####################################################



    model_path = f'./{save_path}/model_{timestamp}_{model.__class__.__name__}_last'
    torch.save(model.state_dict(), model_path)

    print(acc_val.compute())
    return model, best_model_path

In [15]:
DATA_FOLDER_PATH="./data/"
LIST_LABEL = label_dict_from_config_file("hand_gesture.yaml")
train_path = os.path.join(DATA_FOLDER_PATH,"landmark_train.csv")
val_path = os.path.join(DATA_FOLDER_PATH,"landmark_val.csv")
save_path = './models'
os.makedirs(save_path,exist_ok=True)

trainset = CustomImageDataset(train_path)
################## Q3: Khởi tạo DataLoader cho training ################## Q3
'''Hoàn thành code để thực hiện khởi tạo DataLoader cho trainset với
batch_size 40 và cho phép xáo trộn
'''
trainloader = torch.utils.data.DataLoader(
    trainset,
    batch_size=40,
    shuffle=True
)
####################################################

valset = CustomImageDataset(os.path.join(val_path))
val_loader = torch.utils.data.DataLoader(valset,batch_size=50, shuffle=False)

##################  ##################
'''Hoàn thành code để thực hiện khởi tạo NeuralNetwork model đã xây dựng ở trên,
khởi tạo hàm loss sử dụng CrossEntropyLoss và khởi tạo early stopper với patience
là 30 và min_delta là 0.01
'''
# Khởi tạo model MLP
model = NeuralNetwork()  # Tạo instance của model đã định nghĩa

# Khởi tạo hàm loss CrossEntropy cho bài toán phân loại
loss_function = nn.CrossEntropyLoss()

# Khởi tạo early stopper với:
# - patience=30: chờ 30 epochs trước khi dừng nếu loss không giảm
# - min_delta=0.01: ngưỡng cải thiện tối thiểu của loss
early_stopper = EarlyStopper(patience=30, min_delta=0.01)
####################################################

################## Q4: Cấu hình Adam optimizer ################## Q4
'''Hoàn thành code để thực hiện cấu hình Adam optimizer cho các tham số của
model với tốc độ học là 0.0001
'''
optimizer = optim.Adam(model.parameters(), lr=0.0001)
####################################################


model, best_model_path = train(trainloader, val_loader, model, loss_function, early_stopper, optimizer)

Loss batch_number == 1: type = <class 'torch.Tensor'> 
 item: 
 1.6262407302856445
Epoch 0: 
Accuracy train:0.18940137326717377, val:0.24232082068920135
LOSS train 1.6148266608898456 valid 1.6017958919207256
Training vs. Validation Loss {'Training': 1.6148266608898456, 'Validation': 1.6017958919207256} 1
Training vs. Validation accuracy {'Training': 0.18940137326717377, 'Validation': 0.24232082068920135} 1
Loss batch_number == 1: type = <class 'torch.Tensor'> 
 item: 
 1.5843403339385986
Epoch 1: 
Accuracy train:0.23846909403800964, val:0.32764506340026855
LOSS train 1.5947096118560204 valid 1.5862596035003662
Training vs. Validation Loss {'Training': 1.5947096118560204, 'Validation': 1.5862596035003662} 2
Training vs. Validation accuracy {'Training': 0.23846909403800964, 'Validation': 0.32764506340026855} 2
Loss batch_number == 1: type = <class 'torch.Tensor'> 
 item: 
 1.6090214252471924
Epoch 2: 
Accuracy train:0.2826300263404846, val:0.3344709873199463
LOSS train 1.5730596368129437

In [16]:
list_label = label_dict_from_config_file("hand_gesture.yaml")
DATA_FOLDER_PATH="./data/"
testset = CustomImageDataset(os.path.join(DATA_FOLDER_PATH,"landmark_test.csv"))

# Test DataLoader instantiation
################## # 1. Khởi tạo DataLoader cho test set ##################
''' Hoàn thành code bên dưới để  khởi tạo DataLoader cho testset with batch size
20, không cho phép shuffle
'''
test_loader = torch.utils.data.DataLoader(
    testset,          # Dataset test
    batch_size=20,    # Kích thước batch
    shuffle=False     # Không xáo trộn dữ liệu test
)

################## #2. Load model và set up prediction ########################
network = NeuralNetwork()
network.load_state_dict(torch.load(best_model_path, weights_only=False)) # Load trọng số tốt nhất

network.eval() # Tắt chế độ train

# 3. Đánh giá trên test set
acc_test = Accuracy(num_classes=len(list_label), task='MULTICLASS')
for i, test_data in enumerate(test_loader):
    test_input, test_label = test_data
    ################## # Thực hiện dự đoán và cập nhật accuracy ################## Q7
    '''Hoàn thành code bên dưới để  predict class của cử chỉ và update accuracy
    với kết quả predict và true labels
    '''
    preds = network.predict_with_known_class(test_input)
    acc_test.update(preds, test_label)

    ####################################################

# 4. In kết quả
print(network.__class__.__name__)
print(f"Accuracy of model:{acc_test.compute().item()}")
print("========================================================================")

NeuralNetwork
Accuracy of model:0.9724409580230713


In [ ]:
import matplotlib.pyplot as plt

# Vẽ biểu đồ huấn luyện và validation loss qua các epoch
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Train vs Validation Loss')
plt.show()

In [17]:
import shutil
from google.colab import files

# Specify the folder path you want to download
path_folder = "/content/models"

# Compress the folder into a zip file
shutil.make_archive('models', 'zip', path_folder)

# Download the zip file
files.download('models.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>